In [4]:
import pandas as pd

In [5]:
from bs4 import BeautifulSoup
import requests
from lxml import html
import json
import numpy as np

# Get Players

In [55]:
#Team Data
dump = requests.get('https://fantasy.premierleague.com/drf/teams')
load = dump.text
team_dump = json.loads(load)

team_list = []

for team in range(0, len(team_dump)):
    team_list.append(team_dump[team])
    
team_df = pd.DataFrame(team_list)
team_df.rename(columns={'code': 'team_code', 'name': 'team_name'}, inplace=True)


In [104]:
# Position Data
dump = requests.get('https://fantasy.premierleague.com/drf/element-types')
load = dump.text
position_dump = json.loads(load)

position_list = []

for pos in range(0, len(position_dump)):
    position_list.append(position_dump[pos])
    
pos_df = pd.DataFrame(position_list)
pos_df.rename(columns={'id': 'element_type'}, inplace=True)

In [33]:
dump = requests.get('https://fantasy.premierleague.com/drf/bootstrap')
load = dump.text
json_dump = json.loads(load)

In [34]:
datapoints = ['id', 'web_name', 'first_name', 'total_points', 'event_points', 'now_cost','cost_change_start', 
              'selected_by_percent', 'form','ep_next', 'team_code', 'element_type',
              'chance_of_playing_next_round','minutes', 'bonus', 'goals_scored', 'assists', 'clean_sheets', 
              'goals_conceded', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'ea_index', 'news']

player_list = []

for player in range(0, len(json_dump['elements'])):
    stats = [json_dump['elements'][player][x] for x in datapoints]
    player_list.append(stats)
    
player_df = pd.DataFrame(player_list)


# Rename columns
player_df.columns = datapoints
player_df['element_type']-=1

In [86]:
team_df.iloc[0]

team_code                                                                3
current_event_fixture    [{u'is_home': True, u'day': 29, u'event_day': ...
draw                                                                     0
form                                                                  None
id                                                                       1
link_url                                                                  
loss                                                                     0
team_name                                                          Arsenal
next_event_fixture       [{u'is_home': False, u'day': 7, u'event_day': ...
played                                                                   0
points                                                                   0
position                                                                 0
short_name                                                             ARS
strength                 

# Join the dataframes

In [105]:
combined_df_first = pd.merge(player_df, team_df, on='team_code', how='inner')
combined_df = pd.merge(combined_df_first, pos_df, on='element_type', how='left')

# Get Positions

In [6]:
players = requests.get('https://fantasy.premierleague.com/player-list/')
totals = players.text
total_soup = BeautifulSoup(totals)

In [7]:
#Get all player names

player_names = total_soup.find_all('td')[::4]

players = []

for player in player_names:
    name = player.get_text()
    players.append(name)

player_name_df = pd.DataFrame({"Name": players})

In [8]:
# Positions for first and last player in each field

for i, e in enumerate(player_name_df['Name']):
    if e == 'De Gea':   
        gk=i
    if e == 'Azpilicueta':   
        defe=i
    if e == 'Salah':   
        mid=i
    if e == 'Kane':   
        att=i

In [9]:
#Populate df with positions
player_name_df['position']=''


player_name_df.loc[gk:(defe),'position']='Goalkeeper'
player_name_df.loc[defe:(mid),'position']='Defender'
player_name_df.loc[mid:(att),'position']='Midfielder'
player_name_df.loc[att:,'position']='Attacker'

In [10]:
fpl_players =  pd.merge(player_name_df, player_df, left_on='Name', right_on='web_name')

In [11]:
# Deal with duplicates

duplicats = fpl_players['Name'].value_counts()>1

dup_names = fpl_players['Name'].isin(['Rico', 'McCarthy','Ward','Mendy', 'Bennett','Davies','Gray','Long','Simpson','Reid','Sánchez','McCartyh','Stephens', 'Pereira', 'Murphy', 'Williams'])
dup_ids = fpl_players['id'].isin([295, 359])


fpl_players.drop(fpl_players[dup_names].index, inplace=True)
fpl_players.drop(fpl_players[dup_ids].index, inplace=True)

/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/algorithms.py:410: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  f = lambda x, y: htable.ismember_object(x, values)
/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


# Optimal Choice

In [12]:
import pulp
import regex
import pickle
import itertools

# Iteration 3.0

In [13]:
#Change format of df
fpl_players['influence_f'] = fpl_players['influence'].astype(float)
fpl_players['influence_f'].fillna(0, inplace=True)

fpl_players['bps_f'] = fpl_players['bps'].astype(float)
fpl_players['bps_f'].fillna(0, inplace=True)

fpl_players['now_cost_f'] = fpl_players['now_cost'].astype(float)
fpl_players['now_cost'].fillna(0, inplace=True)

fpl_players['ict_f'] = fpl_players['ict_index'].astype(float)
fpl_players['ict_f'].fillna(0, inplace=True)

#Make variables

#Player variables by position
influence = fpl_players[['id',"influence_f"]]
threat = fpl_players[['id',"threat"]]
creativity = fpl_players[['id',"creativity"]]

# Get weighted summary stats for each player

condlist = [fpl_players['position']=='Goalkeeper', fpl_players['position']=='Defender',fpl_players['position']=='Midfielder',fpl_players['position']=='Attacker']
choicelist = [fpl_players['stats']=fpl_players['influence'],fpl_players['stats']=fpl_players['influence'],fpl_players['stats']=fpl_players['influence'],fpl_players['stats']=fpl_players['influence']]

In [14]:
#Make variables for positions
sorting = 'influence_f'
top_players = 100


gks = fpl_players[fpl_players['position']=='Goalkeeper'].sort_values(by=sorting, ascending=False)[:20]
defs = fpl_players[fpl_players['position']=='Defender'].sort_values(by=sorting, ascending=False)[:20]
mds = fpl_players[fpl_players['position']=='Midfielder'].sort_values(by=sorting, ascending=False)[:20]
ats = fpl_players[fpl_players['position']=='Attacker'].sort_values(by=sorting, ascending=False)[:20]

rgks=range(0, len(gks))
rdefs=range(0, len(defs))
rmds=range(0, len(mds))
rats=range(0, len(ats))

In [15]:
#Make tuples


gk = [tuple(c) for c in itertools.combinations(gks['id'], 2)]
df = [tuple(c) for c in itertools.combinations(defs['id'], 5)]
md = [tuple(c) for c in itertools.combinations(mds['id'], 5)]
at = [tuple(c) for c in itertools.combinations(ats['id'], 3)]



# Possible tuples
possible_gk = [tuple(c) for c in pulp.allcombinations(gks['id'], 2)]
possible_df = [tuple(c) for c in pulp.allcombinations(defs['id'], 2)]
possible_md = [tuple(c) for c in pulp.allcombinations(mds['id'], 5)]
possible_at = [tuple(c) for c in pulp.allcombinations(ats['id'], 3)]

In [16]:
#Make optjection function

def total_influence(player, var):
    """Find the value of the table - 
    by calculating the combined influence between the players"""
    u = []
    
    #for i in player:
        #vol = fpl_players[fpl_players['id']==i][var].item()
        #u.append(vol)

    for i in range(0, len(player)):
        vol = fpl_players[fpl_players['id']==player[i]][var].item()
        u.append(vol)
    return sum(u)


In [17]:
#Make price function

def total_price(player):
    """Find the current price of the player """
    u = []
    
    for i in range(0, len(player)):
        vol = fpl_players[fpl_players['id']==player[i]]['now_cost'].item()
        u.append(vol)
    return sum(u)


In [18]:
# Make variables

# for all possible GK options
g = pulp.LpVariable.dicts('gks', gk, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# for all possible DF options
d = pulp.LpVariable.dicts('dfs', df, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# for all possible MD options
m = pulp.LpVariable.dicts('mds', md, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# for all possible AT options
a = pulp.LpVariable.dicts('ats', at, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

In [19]:
#Optimization goal

player_selection = pulp.LpProblem("Player selection model", pulp.LpMaximize)

#player_selection += [total_influence(keepers) * x[keepers] for keepers in possible_gk]
player_selection += pulp.lpSum([total_influence(player,'influence_f') * g[player] for player in gk]
                              +[total_influence(player,'influence_f') * d[player] for player in df] 
                              +[total_influence(player,'influence_f') * m[player] for player in md]
                              +[total_influence(player,'influence_f') * a[player] for player in at]), 'Max influence'

In [20]:
# Get right number of sets, 1 for each position
player_selection += pulp.lpSum([g[player] for player in gk]) == 1, "Maximum_number_of_gks"
player_selection += pulp.lpSum([d[player] for player in df]) == 1, "Maximum_number_of_dfs"
player_selection += pulp.lpSum([m[player] for player in md]) == 1, "Maximum_number_of_mds"
player_selection += pulp.lpSum([a[player] for player in at]) == 1, "Maximum_number_of_ats"

In [21]:
#Budget constraint

player_selection += pulp.lpSum([total_price(player) * g[player] for player in gk]
                              +[total_price(player) * d[player] for player in df] 
                              +[total_price(player) * m[player] for player in md]
                              +[total_price(player) * a[player] for player in at]) <= 1000, 'Max influence'

# Model Result

In [22]:
player_selection.solve()

1

In [23]:
pulp.LpStatus[player_selection.status]

'Optimal'

In [57]:
# Total Cost
cost = 0
for team in gk:
    if g[team].value() == 1.0:
        for x in range(0,2):
            cost += fpl_players[fpl_players['id']==team[x]][['now_cost']].values
for team in df:
    if d[team].value() == 1.0:
        for x in range(0, 5):
            cost += fpl_players[fpl_players['id']==team[x]][['now_cost']].values
for team in md:
    if m[team].value() == 1.0:
        for x in range(0, 5):
            cost += fpl_players[fpl_players['id']==team[x]][['now_cost']].values
for team in at:
    if a[team].value() == 1.0:
        for x in range(0,3):
             cost += fpl_players[fpl_players['id']==team[2]][['now_cost']].values
                
print "Total cost of the team selection: " + str(cost[0][0])

Total cost of the team selection: 925


In [24]:
for team in gk:
    if g[team].value() == 1.0:
        print(team)
        print fpl_players[fpl_players['id']==team[0]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[-1]][['Name', 'now_cost']]

(490, 88)
    Name  now_cost
22  Hart        45
         Name  now_cost
18  Etheridge        46


In [59]:
for team in df:
    if d[team].value() == 1.0:
        print(team)
        print fpl_players[fpl_players['id']==team[0]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[1]][['Name', 'now_cost']]  
        print fpl_players[fpl_players['id']==team[2]][['Name', 'now_cost']]  
        print fpl_players[fpl_players['id']==team[3]][['Name', 'now_cost']] 
        print fpl_players[fpl_players['id']==team[4]][['Name', 'now_cost']]  

(72, 49, 27, 378, 7)
          Name  now_cost
137  Tarkowski        50
      Name  now_cost
170  Duffy        45
           Name  now_cost
167  Steve Cook        46
        Name  now_cost
157  Holebas        49
        Name  now_cost
112  Mustafi        54


In [26]:
for team in md:
    if m[team].value() == 1.0:
        print(team)
        print fpl_players[fpl_players['id']==team[0]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[1]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[2]][['Name', 'now_cost']]  
        print fpl_players[fpl_players['id']==team[3]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[4]][['Name', 'now_cost']] 

(122, 40, 251, 233, 302)
       Name  now_cost
276  Hazard       110
       Name  now_cost
320  Fraser        59
     Name  now_cost
278  Mané       100
         Name  now_cost
299  Maddison        68
      Name  now_cost
287  Pogba        82


In [27]:
for team in at:
    if a[team].value() == 1.0:
        print(team)
        print fpl_players[fpl_players['id']==team[0]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[1]][['Name', 'now_cost']]
        print fpl_players[fpl_players['id']==team[2]][['Name', 'now_cost']]  

(280, 480, 258)
       Name  now_cost
475  Agüero       114
         Name  now_cost
485  Mitrovic        70
     Name  now_cost
508  Ings        57


In [52]:
fpl_players[fpl_players['id']==495]

,Name,position,id,web_name,first_name,total_points,event_points,now_cost,cost_change_start,selected_by_percent,...,influence,creativity,threat,ict_index,ea_index,news,influence_f,bps_f,now_cost_f,ict_f
496,Vydra,Attacker,495,Vydra,Matej,0,0,60,0,0.1,...,0.0,0.0,0.0,0.0,0,,0.0,0.0,60.0,0.0


# Get output

In [1101]:
player_choice.solve()

1

In [1129]:
for v in player_choice.variables():
    if v.varValue > 0:
        print(fpl_players[fpl_players['id']==int(v.name.split('_')[-1])]['position'],":",
            fpl_players[fpl_players['id']==int(v.name.split('_')[-1])]['Name'], "=", v.varValue)
    else:
        continue
